In [6]:
import time
from pytket.circuit import Circuit, CircBox, OpType, QControlBox
from pytket.extensions.qiskit import AerBackend
from pytket.passes import DecomposeBoxes, SynthesiseTket

In [7]:
backend = AerBackend()

from pytket.circuit import Circuit, CircBox
import numpy as np

def build_qft_circuit(n_qubits: int, do_swaps: True) -> Circuit:
    circ = Circuit(n_qubits, name="QFT")
    for i in range(n_qubits):
        circ.H(i)
        for j in range(i + 1, n_qubits):
            circ.CU1(1 / 2 ** (j - i), j, i)
    if do_swaps:
        for k in range(0, n_qubits // 2):
            circ.SWAP(k, n_qubits - k - 1)
    return circ

class DraperQFTAdderConstantPytket:
    def __init__(self, num_state_qubits: int, constant: int) -> Circuit:
        circuit = Circuit(num_state_qubits)

        qft_circuit = build_qft_circuit(num_state_qubits, do_swaps=False)
        circuit.add_gate(CircBox(qft_circuit),[k for k in range(num_state_qubits)])

        # Apply phase rotations for the constant
        for qubit in range(num_state_qubits):
            angle = (constant % (2 ** (qubit + 1))) * np.pi / (2 ** qubit)
            circuit.Rz(angle, qubit)

        # Apply inverse QFT
        circuit.add_gate(CircBox(qft_circuit).dagger,[k for k in range(num_state_qubits)])

        self.circuit = circuit

    def get_circuit(self) -> Circuit:
        return self.circuit

In [8]:
def get_reflect_around_zero(size):
    qc = Circuit(size, name="reflection")
    qc.X(0)
    qc.H(0)
    qc.add_gate(QControlBox(CircBox(Circuit(1).X(0)), n_controls = size-1, control_state=[0]*(size-1)), [k for k in range(size)])
    qc.H(0)
    qc.X(0)

    return qc

def get_cir_be(qc, data, block):
    qc.H(block[0])
    qc.H(block[2])
    adder_2_cir = DraperQFTAdderConstantPytket(len(data)+1, 2).get_circuit()
    adder_1_cir = DraperQFTAdderConstantPytket(len(data)+1, -1+2**(len(data)+1)).get_circuit()
    qc.add_gate(QControlBox(CircBox(adder_2_cir), n_controls = 1, control_state=0), [block[0]]+[data[k] for k in range(len(data))]+[block[1]])
    qc.add_gate(CircBox(adder_1_cir), [data[k] for k in range(len(data))]+[block[1]])
    qc.add_gate(QControlBox(CircBox(get_reflect_around_zero(len(data))),n_controls=1, control_state=0),[block[2]]+[data[k] for k in range(len(data))])
    qc.H(block[0])
    qc.H(block[2])

    return qc

def apply_projector_controlled_phase(qc, phase, block_reg, aux_reg):
    qc.add_gate(QControlBox(CircBox(Circuit(1).X(0)), n_controls = len(block_reg), control_state=[0]*len(block_reg)),
                                         [block_reg[k] for k in range(len(block_reg))] + [aux_reg[0]]
                                         )
    qc.Rz(phase, aux_reg[0])

    qc.add_gate(QControlBox(CircBox(Circuit(1).X(0)), n_controls = len(block_reg), control_state=[0]*len(block_reg)),
                                         [block_reg[k] for k in range(len(block_reg))] + [aux_reg[0]]
                                         )


def apply_qsvt_step(qc, phase1, phase2, u, data, block, qsvt_aux):

    qc.add_gate(CircBox(u), [data[l] for l in range(len(data))]+ [block[l] for l in range(len(block))])
    apply_projector_controlled_phase(qc, phase1, block, qsvt_aux)
    qc.add_gate(CircBox(u).dagger, [data[l] for l in range(len(data))]+ [block[l] for l in range(len(block))])
    apply_projector_controlled_phase(qc, phase2, block, qsvt_aux)


def get_qsvt_circuit(qsvt_phases,
            size):

    qsvt_cir = Circuit()
    data = qsvt_cir.add_q_register("data", size)
    block = qsvt_cir.add_q_register("block", 3)
    cir_be = Circuit()
    data = cir_be.add_q_register("data", size)
    block = cir_be.add_q_register("block", 3)

    qsvt_aux = qsvt_cir.add_q_register("qsvt_aux", 1)
    cir_be = get_cir_be(cir_be,data, block)

    qsvt_cir.H(qsvt_aux[0])
    apply_projector_controlled_phase(qsvt_cir, qsvt_phases[0], block, qsvt_aux)
    for i in range(int(np.floor((len(qsvt_phases) - 1) / 2))):
        apply_qsvt_step(qsvt_cir,
                        qsvt_phases[(2 * i) + 1], qsvt_phases[(2 * i) + 2],
                        cir_be,
                        data,
                        block,
                        qsvt_aux
                       )
    qsvt_cir.add_gate(CircBox(cir_be), [data[l] for l in range(len(data))]+ [block[l] for l in range(len(block))])
    apply_projector_controlled_phase(qsvt_cir, qsvt_phases[len(qsvt_phases) - 1], block, qsvt_aux)
    qsvt_cir.H(qsvt_aux[0])

    return qsvt_cir

In [9]:
SIZES = list(range(5, 21))
DEGREE = 3
QSVT_PHASES = [1.280311896404252, 8.127145628464149, 1.8439603212845617, -5.002873410775335]
save_dict = {}
import pandas as pd


for SIZE in SIZES:

    start_time = time.time()
    qc_qsvt = get_qsvt_circuit(QSVT_PHASES, SIZE)
    DecomposeBoxes().apply(qc_qsvt)
    SynthesiseTket().apply(qc_qsvt)
    compiled_circ = backend.get_compiled_circuit(qc_qsvt)
    transpilation_time = time.time()-start_time
    depth = compiled_circ.depth()
    cx_counts = compiled_circ.n_gates_of_type(OpType.CX)
    width = compiled_circ.n_qubits

    print(f"==== tket for {SIZE}==== time: {transpilation_time}")
    print(f"depth: {depth}")
    print(f"cx_counts: {cx_counts}")
    print(f"width: {width}")
    print(compiled_circ)

    save_dict[SIZE] = {"time": transpilation_time, "depth": depth, "cx_counts": cx_counts, "width": width}
    pd.DataFrame(save_dict).to_csv("tket_classiq_qsvt.csv")


==== tket for 5==== time: 15.530531883239746
depth: 1714
cx_counts: 1284
width: 9
<tket::Circuit, qubits=9, gates=2631>
==== tket for 6==== time: 21.24307680130005
depth: 2203
cx_counts: 1781
width: 10
<tket::Circuit, qubits=10, gates=3637>
==== tket for 7==== time: 31.844398975372314
depth: 2765
cx_counts: 2351
width: 11
<tket::Circuit, qubits=11, gates=4786>
==== tket for 8==== time: 38.528746128082275
depth: 3437
cx_counts: 3011
width: 12
<tket::Circuit, qubits=12, gates=6105>
==== tket for 9==== time: 52.666861057281494
depth: 4222
cx_counts: 3764
width: 13
<tket::Circuit, qubits=13, gates=7629>
==== tket for 10==== time: 66.05526494979858
depth: 5045
cx_counts: 4582
width: 14
<tket::Circuit, qubits=14, gates=9274>
==== tket for 11==== time: 83.20756387710571
depth: 5992
cx_counts: 5509
width: 15
<tket::Circuit, qubits=15, gates=11130>
==== tket for 12==== time: 98.93545198440552
depth: 6967
cx_counts: 6492
width: 16
<tket::Circuit, qubits=16, gates=13112>
==== tket for 13==== time